<a href="https://colab.research.google.com/github/ismael-rtellez/Tensorflow_Series_Assignments/blob/main/TensorFlow_Sprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorFlow Series Assignment

### Problem 1: Looking back on scracth

Looking back at the scratchesso far, when using scratch implementation of neural networks had to manually implement:

1.  Weight initialization
2.  I had to define forward propagation
3.  I needed to calculate loss, including mean squared error and cross-entropy
4.  I implementted backpropagation
5.  Implemented gradient parameter updates
6.  I needed and epoch loop
7.  I implemented Mini-batch processing
8.  I also did accuracy evaluation
9.  I also performed preprocessing of data load, split, and normalize

### Problem 2: Consider the correspondence between scracth and TensorFlow


This is how TensorFlow implements tjhe following:
1.  Weight initialization - uses ```tf.variable(tf.random_normal(...))```
2.  Forward propagation - done via ```tf.matmul```, ```tf.add```, ```tf.nn.relu```
3.  Loss calculation - achieved by ```tf.nn.sigmoid_cross_entropy_with_logits```
4.  Backpropagation - one by ```optimizer.minimize(loss_op)```
5.  Parameter update(gradient) - achieved by ```train_op = optimizer.minimize(...)```
6.  Epoch loop - donne via ```for epoch in range(num_epochs)```
7.  Mini-batch handling - is ```GetMiniBatch``` class with ```for x, y in ...```
8.  Accuracy calculation - implemented via ```tf.equal(...)```, ``````tf.reduce_mean(...)
9.  Data preprocessing - achieved by ```pandas```, ```numpy```, and ```sklearn```

### Problem 3: Create a model of Iris using all three types of objective variables

In [ ]:
# Create a model of Iris using all three types objective variable
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Loading Iris dataset
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("Iris.csv")
X = df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = df["Species"].values

# One-hot encode labels
encoder = LabelBinarizer()
y = encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_input = X_train.shape[1]
n_hidden1, n_hidden2 = 50, 100
n_classes = 3
n_samples = X_train.shape[0]

# Placeholder
X_ph = tf.placeholder(tf.float32, [None, n_input])
y_ph = tf.placeholder(tf.float32, [None, n_classes])

# Weights and biases
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
    'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
    'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden1])),
    'b2': tf.Variable(tf.random_normal([n_hidden2])),
    'b3': tf.Variable(tf.random_normal([n_classes]))
}

# Model
def model(x):
    l1 = tf.nn.relu(tf.add(tf.matmul(x, weights['w1']), biases['b1']))
    l2 = tf.nn.relu(tf.add(tf.matmul(l1, weights['w2']), biases['b2']))
    return tf.matmul(l2, weights['w3']) + biases['b3']

logits = model(X_ph)

# Loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_ph, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y_ph, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

# Training
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        for i in range(0, n_samples, batch_size):
            x_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            sess.run(optimizer, feed_dict={X_ph: x_batch, y_ph: y_batch})

        if epoch % 10 == 0:
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X_ph: X_val, y_ph: y_val})
            print(f"Epoch {epoch}, Val_Loss: {val_loss:.4f}, Val_Acc: {val_acc:.3f}")

    test_acc = sess.run(accuracy, feed_dict={X_ph: X_test, y_ph: y_test})
    print("Test Accuracy: ", test_acc)



Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Saving Iris.csv to Iris.csv
Epoch 0, Val_Loss: 133.8760, Val_Acc: 0.417
Epoch 10, Val_Loss: 0.2684, Val_Acc: 0.875
Epoch 20, Val_Loss: 0.4919, Val_Acc: 0.875
Epoch 30, Val_Loss: 0.2160, Val_Acc: 0.958
Epoch 40, Val_Loss: 0.2023, Val_Acc: 0.958
Epoch 50, Val_Loss: 0.2264, Val_Acc: 0.917
Epoch 60, Val_Loss: 0.3029, Val_Acc: 0.917
Epoch 70, Val_Loss: 0.3538, Val_Acc: 0.917
Epoch 80, Val_Loss: 0.3831, Val_Acc: 0.917
Epoch 90, Val_Loss: 0.3891, Val_Acc: 0.917
Test Accuracy:  1.0


### Problem 4: Create a model for House Prices

In [ ]:
# Create a model of house prices
# Loading dataset
df1 = pd.read_csv("train.csv")
df1 = df1[["GrLivArea", "YearBuilt", "SalePrice"]].dropna()
X1 = df1[["GrLivArea", "YearBuilt"]].values
y1 = df1["SalePrice"].values.reshape(-1, 1)

# Normalizing
X1 = (X1 - X1.mean(axis=0)) / X1.std(axis=0)
y1 = (y1 - y1.mean()) / y1.std()

# Split data
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=0)
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_train1, y_train1, test_size=0.2, random_state=0)

# Placeholder
X1_ph = tf.placeholder(tf.float32, [None, 2])
y1_ph = tf.placeholder(tf.float32, [None, 1])

# Weights and biases
W1 = tf.Variable(tf.random.normal([2, 64]))
b1 = tf.Variable(tf.zeros([64]))
W2 = tf.Variable(tf.random.normal([64, 1]))
b2 = tf.Variable(tf.zeros([1]))

# Model
l1 = tf.nn.relu(tf.matmul(X1_ph, W1) + b1)
output = tf.matmul(l1, W2) + b2

# Loss and optimizer
loss = tf.reduce_mean(tf.square(output - y1_ph))
optimizer1 = tf.train.AdamOptimizer(0.01).minimize(loss)

# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        for i in range(0, len(X_train1), batch_size):
            x_batch1 = X_train1[i:i+batch_size]
            y_batch1 = y_train1[i:i+batch_size]
            sess.run(optimizer1, feed_dict={X1_ph: x_batch1, y1_ph: y_batch1})

        if epoch % 10 == 0:
            val_loss1 = sess.run(loss, feed_dict={X1_ph: X_val1, y1_ph: y_val1})
            print(f"Epoch {epoch}, Val_Loss: {val_loss1:.4f}")

    test_loss = sess.run(loss, feed_dict={X1_ph: X_test1, y1_ph: y_test1})
    print("Test Loss: ", test_loss)

Epoch 0, Val_Loss: 0.3246
Epoch 10, Val_Loss: 0.2805
Epoch 20, Val_Loss: 0.2564
Epoch 30, Val_Loss: 0.2641
Epoch 40, Val_Loss: 0.2686
Epoch 50, Val_Loss: 0.2565
Epoch 60, Val_Loss: 0.2634
Epoch 70, Val_Loss: 0.2436
Epoch 80, Val_Loss: 0.2339
Epoch 90, Val_Loss: 0.2323
Test Loss:  0.38721147


### Problem 5: Create an MNIST model

In [ ]:
# Create a model of MNIST
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Loading MNIST from tf.keras
from tensorflow.keras.datasets import mnist
(x_train2, y_train2), (x_test2, y_test2) = mnist.load_data()

# Normalizing and reshaping
x_train2 = x_train2.reshape(-1, 784) / 255.0
x_test2 = x_test2.reshape(-1, 784) / 255.0

# One-hot encode labels
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)
y_train2 = enc.fit_transform(y_train2.reshape(-1, 1))
y_test2= enc.transform(y_test2.reshape(-1, 1))

# Splitting validation set
x_val2= x_train2[-5000:]
y_val2= y_train2[-5000:]
x_train2 = x_train2[:-5000]
y_train2 = y_train2[:-5000]

# Placeholder
X_mnist = tf.placeholder(tf.float32, [None, 784])
y_mnist = tf.placeholder(tf.float32, [None, 10])

# Model parameters
W1_mnist = tf.Variable(tf.random.normal([784, 128]))
b1_mnist = tf.Variable(tf.zeros([128]))
W2_mnist = tf.Variable(tf.random.normal([128, 10]))
b2_mnist = tf.Variable(tf.zeros([10]))

# Building model
l1_mnist = tf.nn.relu(tf.matmul(X_mnist, W1_mnist) + b1_mnist)
logits_mnist = tf.matmul(l1_mnist, W2_mnist) + b2_mnist

# Loss and optimizer
loss_op_mnist = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_mnist, logits=logits_mnist))
optimizer_mnist = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_op_mnist)

# Accuracy
correct_pred_mnist = tf.equal(tf.argmax(logits_mnist, 1), tf.argmax(y_mnist, 1))
accuracy_mnist = tf.reduce_mean(tf.cast(correct_pred_mnist, tf.float32))

# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        for i in range(0, len(x_train2), 100):
            x_batch2 = x_train2[i:i+100]
            y_batch2 = y_train2[i:i+100]
            sess.run(optimizer_mnist, feed_dict={X_mnist: x_batch2, y_mnist: y_batch2})

        val_acc_mnist = sess.run(accuracy_mnist, feed_dict={X_mnist: x_val2, y_mnist: y_val2})
        print(f"Epoch {epoch}, Validation Accuracy: {val_acc_mnist:.4f}")

    test_acc_mnist = sess.run(accuracy_mnist, feed_dict={X_mnist: x_test2, y_mnist: y_test2})
    print("Test Accuracy: ", test_acc_mnist)

Epoch 0, Validation Accuracy: 0.8242
Epoch 1, Validation Accuracy: 0.8720
Epoch 2, Validation Accuracy: 0.8934
Epoch 3, Validation Accuracy: 0.9092
Epoch 4, Validation Accuracy: 0.9194
Epoch 5, Validation Accuracy: 0.9236
Epoch 6, Validation Accuracy: 0.9290
Epoch 7, Validation Accuracy: 0.9316
Epoch 8, Validation Accuracy: 0.9340
Epoch 9, Validation Accuracy: 0.9348
Test Accuracy:  0.9244
